In [2]:
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px



In [3]:
df_new = pd.read_excel('All_Data copy.xls', sheet_name = '6mm')
df = pd.read_csv('2mm.csv')
def process(num):
    if num%18 > 17:
        return (num%18)-18
    else:
        return num%18
def create_new_energy():
    yo = [process(x) for x in df[df['Energy']>18]['Energy']]
    yo = [x if x >= -0.5 and x < 2 else 0 for x in yo]
    a = []
    counter = 1
    for x in range(len(yo)-1):
        if yo[x]!= 0:
            a.append(counter)
            if yo[x]!=0 and yo[x+1]==0:
                counter = counter + 1
        else:
            a.append(0)
    a.append(0)   
    return a 
len(create_new_energy())

6916

In [4]:
def clusterise(df):
    df = df[df['Energy']>18]
    df = df.drop(['Energy'], axis = 1)
    df['Energy']= create_new_energy()
    deletion = []
    for index, row in df.iterrows():
        if row[-1] == 0:
            deletion.append(row.name)
    df = df.drop(labels = deletion)
    df = df.groupby('Energy').max()
    return df

In [5]:
new_df =clusterise(df)

In [6]:
new_df.index

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
            52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
            69],
           dtype='int64', name='Energy')

In [7]:
new_df['13']

Energy
1     254
2      77
3      69
4      71
5      44
     ... 
65      2
66      2
67      2
68      2
69      1
Name: 13, Length: 69, dtype: int64

In [8]:
new_df.T

Energy,1,2,3,4,5,6,7,8,9,10,...,60,61,62,63,64,65,66,67,68,69
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10.2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10.3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10.4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10.6,1,0,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
10.7,0,0,0,1,1,0,1,0,0,1,...,0,1,0,1,1,0,0,0,0,0
10.8,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
10.9,1,0,0,1,1,0,0,1,1,1,...,0,0,0,0,1,0,0,0,0,1


In [9]:
new_df.T.index


Index(['10', '10.1', '10.2', '10.3', '10.4', '10.5', '10.6', '10.7', '10.8',
       '10.9', '11', '11.1', '11.2', '11.3', '11.4', '11.5', '11.6', '11.7',
       '11.8', '11.9', '12', '12.1', '12.2', '12.3', '12.4', '12.5', '12.6',
       '12.7', '12.8', '12.9', '13', '13.1', '13.2', '13.3', '13.4', '13.5',
       '13.6', '13.7', '13.8', '13.9', '14', '14.1', '14.2', '14.3', '14.4',
       '14.5'],
      dtype='object')

In [12]:
df_new = pd.ExcelFile('All_Data copy.xls')
df_new.sheet_names

['2mm', '6mm', '10mm', '15mm', '20mm', '25mm', '30mm', '35mm']

In [25]:
df_new.parse('2mm')

,Mass,10.0,10.1,10.2,10.3,10.4,10.5,10.6,10.7,10.8,...,13.6,13.7,13.8,13.9,14.0,14.1,14.2,14.3,14.4,14.5
0,6.168828,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7305,1254.713570,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,3,2,0
7306,1255.033161,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,3
7307,1255.352792,0,0,0,0,0,0,0,0,0,...,0,0,2,0,2,0,0,0,0,0
7308,1255.672465,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1


In [18]:
sheet_to_df_map = {}
for sheet_name in df_new.sheet_names:
    sheet_to_df_map[sheet_name] = df_new.parse(sheet_name)


{'2mm':              Mass  10.0  10.1  10.2  10.3  10.4  10.5  10.6  10.7  10.8  ...  \
 0        6.168828     0     0     0     0     0     0     0     0     0  ...   
 1        6.188976     0     0     0     0     0     0     0     0     0  ...   
 2        6.166552     0     0     0     0     0     0     0     0     0  ...   
 3        6.166552     0     0     0     0     0     0     0     0     0  ...   
 4        6.188976     0     0     0     0     0     0     0     0     0  ...   
 ...           ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
 7305  1254.713570     0     0     0     0     0     0     0     0     0  ...   
 7306  1255.033161     0     0     0     0     0     0     0     0     0  ...   
 7307  1255.352792     0     0     0     0     0     0     0     0     0  ...   
 7308  1255.672465     0     0     0     0     0     0     0     0     0  ...   
 7309  1255.992178     0     0     0     0     0     0     0     0     0  ...   
 
       13.6  13.7  

In [19]:
sheet_to_df_map

{'2mm':              Mass  10.0  10.1  10.2  10.3  10.4  10.5  10.6  10.7  10.8  ...  \
 0        6.168828     0     0     0     0     0     0     0     0     0  ...   
 1        6.188976     0     0     0     0     0     0     0     0     0  ...   
 2        6.166552     0     0     0     0     0     0     0     0     0  ...   
 3        6.166552     0     0     0     0     0     0     0     0     0  ...   
 4        6.188976     0     0     0     0     0     0     0     0     0  ...   
 ...           ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
 7305  1254.713570     0     0     0     0     0     0     0     0     0  ...   
 7306  1255.033161     0     0     0     0     0     0     0     0     0  ...   
 7307  1255.352792     0     0     0     0     0     0     0     0     0  ...   
 7308  1255.672465     0     0     0     0     0     0     0     0     0  ...   
 7309  1255.992178     0     0     0     0     0     0     0     0     0  ...   
 
       13.6  13.7  

In [27]:
sheet_to_df_map['2mm']['Mass']

0          6.168828
1          6.188976
2          6.166552
3          6.166552
4          6.188976
           ...     
7305    1254.713570
7306    1255.033161
7307    1255.352792
7308    1255.672465
7309    1255.992178
Name: Mass, Length: 7310, dtype: float64

In [48]:
Data = pd.ExcelFile('All_Data copy.xls')

In [129]:
Sheet_names = Data.sheet_names
Sheet_names

['2mm', '6mm', '10mm', '15mm', '20mm', '25mm', '30mm', '35mm']

In [130]:
two_mm = Data.parse('2mm')
six_mm = Data.parse('6mm')
ten_mm = Data.parse('10mm')

In [131]:
columns = two_mm.columns 
a = ["Mass"]
two_mm.columns = a+['2mm '+x for x in columns[1:]]

In [132]:
two_mm

,Mass,2mm 10.0,2mm 10.1,2mm 10.2,2mm 10.3,2mm 10.4,2mm 10.5,2mm 10.6,2mm 10.7,2mm 10.8,...,2mm 13.6,2mm 13.7,2mm 13.8,2mm 13.9,2mm 14.0,2mm 14.1,2mm 14.2,2mm 14.3,2mm 14.4,2mm 14.5
0,6.168828,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7305,1254.713570,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,3,2,0
7306,1255.033161,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,3
7307,1255.352792,0,0,0,0,0,0,0,0,0,...,0,0,2,0,2,0,0,0,0,0
7308,1255.672465,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1


In [133]:
for x in Sheet_names[1:]:
    df = Data.parse(x)
    columns = df.columns
    df = df.drop('Mass', axis = 1)
    df.columns =[x +' '+ i for i in columns[1:]]
    two_mm = two_mm.join(df)

,Mass,2mm 10.0,2mm 10.1,2mm 10.2,2mm 10.3,2mm 10.4,2mm 10.5,2mm 10.6,2mm 10.7,2mm 10.8,...,35mm 13.6,35mm 13.7,35mm 13.8,35mm 13.9,35mm 14.0,35mm 14.1,35mm 14.2,35mm 14.3,35mm 14.4,35mm 14.5
0,6.168828,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7305,1254.713570,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7306,1255.033161,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7307,1255.352792,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
7308,1255.672465,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [45]:
two_mm

,Mass,10.0,10.1,10.2,10.3,10.4,10.5,10.6,10.7,10.8,...,13.635,13.735,13.835,13.935,14.035,14.135,14.235,14.335,14.435,14.535
0,6.168828,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6.166552,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6.188976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7305,1254.713570,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7306,1255.033161,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7307,1255.352792,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
7308,1255.672465,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [46]:
two_mm.columns

Index(['Mass', '10.0', '10.1', '10.2', '10.3', '10.4', '10.5', '10.6', '10.7',
       '10.8',
       ...
       '13.635', '13.735', '13.835', '13.935', '14.035', '14.135', '14.235',
       '14.335', '14.435', '14.535'],
      dtype='object', length=376)

In [47]:
len(two_mm.columns)

376

In [72]:
[2]+[3]

[2, 3]